In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests

In [2]:
file_path = '~/Downloads/data/groups.csv'
df = pd.read_csv(file_path)

df

,title,description,pi_urls,people_urls,projects_urls
0,Broadercasting,"<p>looks at broadcasting in a broader way, as ...",NaN,NaN,https://www.media.mit.edu/projects/agent-stori...
1,Counter Intelligence,<p>is focused on developing a digitally connec...,NaN,https://www.media.mit.edu/people/selker/overview/,https://www.media.mit.edu/projects/acoustic-im...
2,Gray Matters,<p>explores how computation and communication ...,NaN,NaN,https://www.media.mit.edu/projects/ambient-dis...
3,Personal Fabrication,"<p>is about making things, and about the futur...",NaN,NaN,https://www.media.mit.edu/projects/alf/overvie...
4,Toys of Tomorrow,<p>explores ways that the digital revolution w...,NaN,NaN,https://www.media.mit.edu/projects/affective-s...
...,...,...,...,...,...
171,Media Lab Research Theme: Future Worlds,<h2><b>The Challenge</b></h2><p><b>How can we ...,NaN,NaN,NaN
172,Media Lab Research Theme: Life with AI,<h2><b>The Challenge&nbsp;</b></h2><p><b>Machi...,NaN,NaN,NaN
173,Media Lab Research Theme: Decentralized Society,<h2><b>The Challenge&nbsp;</b></h2><p><b>Our w...,NaN,NaN,NaN
174,Media Lab Research Theme: Connected Mind + Body,<h2><b>The Challenge&nbsp;</b></h2><p><b>Menta...,NaN,NaN,NaN


In [13]:

pis = df['pi_urls'].unique()
pi_df = pd.DataFrame(pis[1:])
pi_df.rename(columns={0: 'url'}, inplace=True)
pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/rlanger/overview/'}, ignore_index=True)
pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/dnewman/overview/'}, ignore_index=True)

/var/folders/67/4cx23sdx471d8kt2pfsw73700000gn/T/ipykernel_35754/3623007229.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/rlanger/overview/'}, ignore_index=True)
/var/folders/67/4cx23sdx471d8kt2pfsw73700000gn/T/ipykernel_35754/3623007229.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pi_df = pi_df.append({'url': 'https://www.media.mit.edu/people/dnewman/overview/'}, ignore_index=True)


In [14]:
def get_image_url(people_url):
    """
    scrape the media lab website to get the image corresponding to a person

    :param people_url: the url to the person's page on the media lab website
    :return: the url of the person's avatar if it exists, None otherwise
    """
    try:
        response = requests.get(people_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        hero = soup.find('div', {'class': 'hero-content-profile-avatar'})
        style = hero['style']
    except:
        return
    match = re.search('url\((".+")\)', style)
    if match:
        image_url = match.group(0)[5:-2]
        return image_url

In [23]:
def get_bio(people_url):
    try:
        response = requests.get(people_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        bio = soup.findAll('div', {'class': 'main-copy'})[0].get_text()
        return bio
    except:
        return

In [16]:
def get_name(people_url):
    try:
        response = requests.get(people_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        bio = soup.find('h1', {'class': 'hero-content-title'}).get_text()
        return bio
    except:
        return

In [17]:
pi_df['image_url'] = pi_df.apply(lambda x:get_image_url(x['url']), axis=1)

In [24]:
pi_df['bio'] = pi_df.apply(lambda x:get_bio(x['url']), axis=1)

In [20]:
pi_df['title'] = pi_df.apply(lambda x:get_name(x['url']), axis=1)

In [21]:
pi_df.set_index('title', inplace=True)

In [25]:
pi_df.to_csv('pis.csv')

In [32]:
# process active groups
active_df = df[df['pi_urls'].notnull()]

In [39]:
def get_group_image_url(project_url):
    """
    scrape the media lab website to get the image corresponding to a project

    :param project_url: the url to the project on the media lab website
    :return: the url of the project image if it exists, None otherwise
    """
    try:
        response = requests.get(project_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        hero = soup.find('div', {'class': 'hero'})
        style = hero['style']
        print(style)
    except:
        return
    match = re.search('url\((".+")\)', style)
    if match:
        image_url = match.group(0)[5:-2]
        return image_url

In [34]:
def get_url(title: str):
    return f'https://www.media.mit.edu/groups/{title.lower().replace(" ", "-")}/overview/'

active_df['url'] = active_df.apply(lambda x:get_url(x['title']), axis=1)

/var/folders/67/4cx23sdx471d8kt2pfsw73700000gn/T/ipykernel_35754/2149523261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_df['url'] = active_df.apply(lambda x:get_url(x['title']), axis=1)


In [40]:
active_df['image_url'] = pi_df.apply(lambda x:get_group_image_url(x['url']), axis=1)

background-image: linear-gradient(25deg, #007b42  0%,#54be49  100%);
background-image: linear-gradient(25deg, #70208c  0%,#3f46bb  100%);
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: linear-gradient(25deg, #de1923  0%,#f46a21  100%);
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: url("https://dam-prod2.media.mit.edu/thumb/2020/11/20/Screen%20Shot%202020-11-20%20at%2012.44.47%20AM.png.1400x1400.png"); background-position: 22.6432291666667% 84.1039713392789%;
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: linear-gradient(25deg, #323896  0%,#0C8ED5  100%);
background-image: linear-gradient(25deg, #

/var/folders/67/4cx23sdx471d8kt2pfsw73700000gn/T/ipykernel_35754/2573018863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_df['image_url'] = pi_df.apply(lambda x:get_group_image_url(x['url']), axis=1)


In [38]:
active_df

,title,description,pi_urls,people_urls,projects_urls,url,image_url
21,Ethics,<h2>This work&nbsp;is being continued through ...,https://www.media.mit.edu/people/sandy/overview/,https://www.media.mit.edu/people/sandy/overvie...,https://www.media.mit.edu/projects/impossible-...,https://www.media.mit.edu/groups/ethics/overview/,NaN
22,Digital Currency Initiative (DCI),<p>The internet enabled people to easily call ...,https://www.media.mit.edu/people/narula/overview/,https://www.media.mit.edu/people/bforde/overvi...,https://www.media.mit.edu/projects/medrec-arch...,https://www.media.mit.edu/groups/digital-curre...,NaN
51,Center for Bits and Atoms,<p>MIT's Center for Bits and Atoms is an inter...,https://www.media.mit.edu/people/neilg-disable...,https://www.media.mit.edu/people/difran/overvi...,https://www.media.mit.edu/projects/origami-sim...,https://www.media.mit.edu/groups/center-for-bi...,NaN
54,Director's Office,<p>The Director's Office at the Media Lab (E14...,https://www.media.mit.edu/people/rlanger/overv...,https://www.media.mit.edu/people/lieber/overvi...,https://www.media.mit.edu/projects/ai-and-glob...,https://www.media.mit.edu/groups/director's-of...,NaN
59,Program in Media Arts and Sciences,<p>The Program in Media Arts and Sciences (MAS...,https://www.media.mit.edu/people/bv/overview/,https://www.media.mit.edu/people/bv/overview/ ...,https://www.media.mit.edu/projects/tech-tutors...,https://www.media.mit.edu/groups/program-in-me...,NaN
72,Social Machines,"<p><i>As of 2021, the MIT Media Lab's Social M...",https://www.media.mit.edu/people/dkroy/overview/,https://www.media.mit.edu/people/dkroy/overvie...,https://www.media.mit.edu/projects/10000x-more...,https://www.media.mit.edu/groups/social-machin...,NaN
75,Affective Computing,<p>The Affective Computing group creates and e...,https://www.media.mit.edu/people/picard/overview/,https://www.media.mit.edu/people/picard/overvi...,https://www.media.mit.edu/projects/kind-and-gr...,https://www.media.mit.edu/groups/affective-com...,NaN
76,Human Dynamics,<p>The Human Dynamics group uses the tools of ...,https://www.media.mit.edu/people/sandy/overview/,https://www.media.mit.edu/people/sandy/overvie...,https://www.media.mit.edu/projects/3-d-modelin...,https://www.media.mit.edu/groups/human-dynamic...,NaN
78,Lifelong Kindergarten,<p>The Lifelong Kindergarten group develops in...,https://www.media.mit.edu/people/mres/overview/,https://www.media.mit.edu/people/millner/overv...,https://www.media.mit.edu/projects/4chan-and-b...,https://www.media.mit.edu/groups/lifelong-kind...,NaN
80,Opera of the Future,<p>The Opera of the Future group (formerly kno...,https://www.media.mit.edu/people/tod/overview/,https://www.media.mit.edu/people/benb/overview...,https://www.media.mit.edu/projects/advanced-au...,https://www.media.mit.edu/groups/opera-of-the-...,NaN
